In [1]:
import pandas as pd
import datetime as dt

In [2]:
all_dates = pd.DataFrame(pd.date_range(start='2015-01-01', end='2022-03-15'), columns=['date'])

all_dates.head()

,date
0,2015-01-01
1,2015-01-02
2,2015-01-03
3,2015-01-04
4,2015-01-05


In [3]:
interest = pd.read_excel('../ini_data/cbr rates.xlsb', engine = 'pyxlsb')
interest.columns = ['date', 'rate']
interest['date'] = pd.TimedeltaIndex(interest['date'], unit='d') + dt.datetime(1899,12,30)

interest.head()

,date,rate
0,2022-02-28,20.00
1,2022-02-14,9.50
2,2021-12-20,8.50
3,2021-10-25,7.50
4,2021-09-13,6.75


In [4]:
usd = pd.read_excel('../ini_data/usd rates history.xlsb', engine = 'pyxlsb')
usd.columns = ['date', 'usd']
usd['date'] = pd.TimedeltaIndex(usd['date'], unit='d') + dt.datetime(1899,12,30)

usd.head()

,date,usd
0,2015-03-05,61.8745
1,2015-03-06,61.8457
2,2015-03-07,59.9938
3,2015-03-11,60.6649
4,2015-03-12,62.6797


In [5]:
gdp = pd.read_excel('../ini_data/GDP quarterly.xls', engine = 'xlrd')
gdp.columns = ['year', 'quarter', 'gdp']
gdp['date'] = gdp.apply(lambda r: dt.date(int(r.year), int((r.quarter - 1) * 3 + 1), 1), axis=1)
gdp.drop(columns=['year', 'quarter'], inplace=True)
gdp = gdp[['date', 'gdp']]
gdp.date = pd.to_datetime(gdp.date)

gdp.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    44 non-null     datetime64[ns]
 1   gdp     43 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 832.0 bytes


In [6]:
processing_idx = pd.read_excel('../ini_data/индекс цен перерабатывающей промышленности.xlsx', engine='openpyxl')
processing_idx['date'] = processing_idx.apply(lambda r: dt.date(int(r.Год), int(r.Месяц), 1), axis=1)
processing_idx.drop(columns=['Год', 'Месяц'], inplace=True)
processing_idx.rename(columns={'Значение': 'processing_idx'}, inplace=True)
processing_idx = processing_idx[['date', 'processing_idx']]
processing_idx.date = pd.to_datetime(processing_idx.date)

processing_idx.head()

,date,processing_idx
0,2013-01-01,99.5
1,2013-02-01,100.3
2,2013-03-01,99.9
3,2013-04-01,99.9
4,2013-05-01,99.3


In [7]:
oil = pd.read_excel('../ini_data/нефть-brent.xlsx', engine='openpyxl')
oil.rename(columns={"Дата": "date", "Значение": "oil"}, inplace=True)

oil.head()

,date,oil
0,2022-03-14,106.90
1,2022-03-11,112.67
2,2022-03-10,109.33
3,2022-03-09,111.14
4,2022-03-08,127.98


In [8]:
steel = pd.read_csv('../ini_data/HistoricalPrices NYSE American Steel Index.csv', usecols=['Date', ' Close'], parse_dates=['Date'])
steel.rename(columns={'Date': 'date', ' Close': 'steel'}, inplace=True)

steel.head()

,date,steel
0,2022-03-14,1782.36
1,2022-03-11,1842.49
2,2022-03-10,1868.94
3,2022-03-09,1830.73
4,2022-03-08,1815.89


In [9]:
dfs = [interest, usd, gdp, processing_idx, oil, steel]
df = pd.DataFrame(pd.date_range(start='2015-01-01', end='2022-03-15'), columns=['date'])
for right in dfs:
    try:
        df = pd.merge(df, right, on='date', how='left')
    except:
        import pdb; pdb.set_trace()
    
df.head()

,date,rate,usd,gdp,processing_idx,oil,steel
0,2015-01-01,NaN,NaN,18467.912804,103.0,55.84,NaN
1,2015-01-02,NaN,NaN,NaN,NaN,56.42,979.56
2,2015-01-03,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-05,NaN,NaN,NaN,NaN,53.11,939.91


In [10]:
df.to_csv('../ini_data/all_indices.csv')

In [11]:
df.describe()

,rate,usd,gdp,processing_idx,oil,steel
count,35.000000,1728.000000,27.000000,84.000000,2001.000000,1810.000000
mean,8.442857,66.048536,24894.537399,100.692262,58.993989,1096.538011
std,3.248351,6.983933,3963.712630,1.263169,13.939607,321.321260
min,4.250000,49.177700,18467.912804,97.290000,19.330000,450.020000
25%,6.500000,61.459525,22076.344498,99.852500,49.030000,880.637500
50%,7.500000,65.207550,24552.081753,100.590000,59.300000,1084.315000
75%,9.625000,72.616975,27388.838969,101.462500,67.970000,1270.880000
max,20.000000,111.756400,33911.192510,103.600000,127.980000,1927.010000


In [11]:
df.ffill(inplace=True)

In [12]:
df.to_csv('../ini_data/all_indices_all_dates.csv')